In [ ]:
pip install imblearn

In [ ]:
pip install xgboost

In [ ]:
pip install mlxtend

In [ ]:
pip install category_encoders

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from imblearn.pipeline import make_pipeline  # Penting: gunakan imblearn.pipeline!
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import mstats
from sklearn.metrics import roc_auc_score

In [31]:
#df_train = pd.read_csv("train_cleaned.csv")
#df_test = pd.read_csv("test_cleaned (2).csv")

In [3]:
#drop primaryGenreName
df_train = df_train.drop(columns=['primaryGenreName'])
df_test = df_test.drop(columns=['primaryGenreName'])

In [8]:
#one hot encoding primaryGenreName
#from sklearn.preprocessing import OneHotEncoder

#encoder = OneHotEncoder()
#df_train = pd.get_dummies(df_train, columns=['primaryGenreName'])
#df_test = pd.get_dummies(df_test, columns=['primaryGenreName'])

In [32]:
df_test = df_test.drop(columns=['ID'])

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6886 entries, 0 to 6885
Data columns (total 58 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   coppaRisk                                 6886 non-null   bool   
 1   developerCountry_freq                     6886 non-null   int64  
 2   countryCode_freq                          6886 non-null   int64  
 3   userRatingCount_scaled                    6886 non-null   float64
 4   averageUserRating_scaled                  6886 non-null   float64
 5   primaryGenreName_freq                     6886 non-null   int64  
 6   log_min_winsor_scaled                     6886 non-null   float64
 7   log_max_winsor_scaled                     6886 non-null   float64
 8   deviceType_GLOBAL                         6886 non-null   bool   
 9   deviceType_connected-tv/ott               6886 non-null   bool   
 10  deviceType_smartphone               

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 57 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   developerCountry_freq                     3000 non-null   int64  
 1   countryCode_freq                          3000 non-null   int64  
 2   userRatingCount_scaled                    3000 non-null   float64
 3   averageUserRating_scaled                  3000 non-null   float64
 4   primaryGenreName_freq                     3000 non-null   int64  
 5   log_min_winsor_scaled                     3000 non-null   float64
 6   log_max_winsor_scaled                     3000 non-null   float64
 7   deviceType_GLOBAL                         3000 non-null   bool   
 8   deviceType_connected-tv/ott               3000 non-null   bool   
 9   deviceType_smartphone                     3000 non-null   bool   
 10  deviceType_tablet                   

In [2]:
#split data dengan perbandingan 80:20
from sklearn.model_selection import train_test_split

X = df_train.drop(columns=['coppaRisk'])
y = df_train['coppaRisk']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'df_train' is not defined

In [6]:
#split dengan stratified
#from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc

skf = StratifiedKFold(n_splits = 10, #5
                      random_state = 1, 
                      shuffle=True)

rocauc_scores = []

for train_index, test_index in skf.split(X,y):
    X_train = X.iloc[train_index, :]
    X_test = X.iloc[test_index, :]
    y_train = y[train_index]
    y_test = y[test_index]

    #Train the model
    #model.fit(X_train, y_train)

    #Predict on the test set
    #y_pred = model.predict(X_test)

    #Evaluate the model
    rocauc = rocauc_scores(y_test, y_pred)
    rocauc_scores.append(accuracy)
    print(f'Fold ROC AUC: {rocauc}')

# Print average accuracy accross all folds
print(f'\nAverage Accuracy: {sum(rocauc_scores) / len(rocauc_scores)}')

# SMOTE

In [13]:
from imblearn.over_sampling import SMOTE

# Resampling the minority class. The strategy can be changed as required.
sm = SMOTE(sampling_strategy='minority', random_state=42)

# Fit the model to generate the data.
oversampled_X, oversampled_Y = sm.fit_resample(X_train, y_train)

ValueError: could not convert string to float: 'Games'

In [ ]:
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc

skf = StratifiedKFold(n_splits = 10, #5
                      random_state = 42, 
                      shuffle=True)

rocauc_scores = []

for train_index, test_index in skf.split(X,y):
    X_train = X.iloc[train_index, :] #how do i get train_index and val_index?
    X_val = X.iloc[val_index, :] #X.iloc[test_index, :]
    y_train = y[train_index]
    y_test = y[val_index] #y[test_index]

    sm = SMOTE(sampling_strategy='minority', random_state=42)
    
    # Fit the model to generate the data.
    oversampled_X, oversampled_Y = sm.fit_resample(X_train, y_train)

    model = ... # Choose a model here
    model.fit(oversampled_X, oversampled_Y)

    y_pred = model.predict_proba(X_val)


    print(f'For fold {fold}:')
    print(f'Accuracy: {model.score(X_test, y_test)}')
    print(f'f-score: {f1_score(y_test, y_pred)}')

In [15]:
oversampled_Y

0       False
1       False
2       False
3       False
4       False
        ...  
9915     True
9916     True
9917     True
9918     True
9919     True
Name: coppaRisk, Length: 9920, dtype: bool

In [8]:
oversampled_Y.value_counts()

coppaRisk
False    4960
True     4960
Name: count, dtype: int64

# Model Stacking

## Trial 1 (Decision Tree Classifier, Random Forest Classifier, GradientBoostingClassifier)

### Tanpa Handling Imbalanced Class

Hasilnya all 1s bjir

In [6]:

  
# Train the base models  
dt = DecisionTreeClassifier(random_state=42)  
dt.fit(X_train, y_train)  
  
rf = RandomForestClassifier(random_state=42)  
rf.fit(X_train, y_train)  
  
gb = GradientBoostingClassifier(random_state=42)  
gb.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [8]:
# Make predictions on the validation set  
dt_pred = dt.predict(X_val)  
rf_pred = rf.predict(X_val)  
gb_pred = gb.predict(X_val)

In [9]:
from sklearn.linear_model import LogisticRegression
  
# Combine the predictions of the base models into a single feature matrix  
X_val_meta = np.column_stack((dt_pred, rf_pred, gb_pred))  
  
# Train the meta-model on the combined feature matrix and the target values  
meta_model = LogisticRegression(random_state = 42)  
meta_model.fit(X_val_meta, y_val)

LinearRegression()

In [13]:
# Make predictions on new data  
df_test = df_test.drop(columns=['ID'])
dt_pred_new = dt.predict(df_test)  
rf_pred_new = rf.predict(df_test)  
gb_pred_new = gb.predict(df_test)

In [15]:
# Combine the predictions of the base models into a single feature matrix  
X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new))  
  
# Make a prediction using the meta-model  
y_new_pred = meta_model.predict(X_new_meta)    

In [20]:
df_test_result = df_test.copy()
df_test_result['coppaRisk_pred'] = y_new_pred.astype(bool)
#df_test_result['coppaRisk_proba'] = y_new_proba

In [22]:
df_test_result.to_csv('submission.csv', index=False)

### Handling Imbalanced w/ SMOTE

akurasi = 0.64558

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
  
# Train the base models  
dt = DecisionTreeClassifier(random_state=42)  
dt.fit(oversampled_X, oversampled_Y)  
  
rf = RandomForestClassifier(random_state=42)  
rf.fit(oversampled_X, oversampled_Y)  
  
gb = GradientBoostingClassifier(random_state=42)  
gb.fit(oversampled_X, oversampled_Y)

GradientBoostingClassifier(random_state=42)

In [11]:
# Make predictions on the validation set  
dt_pred = dt.predict(X_val)  
rf_pred = rf.predict(X_val)  
gb_pred = gb.predict(X_val)

In [12]:
from sklearn.linear_model import LogisticRegression
  
# Combine the predictions of the base models into a single feature matrix  
X_val_meta = np.column_stack((dt_pred, rf_pred, gb_pred))  
  
# Train the meta-model on the combined feature matrix and the target values  
meta_model = LogisticRegression(random_state = 42)  
meta_model.fit(X_val_meta, y_val)

LogisticRegression(random_state=42)

In [13]:
# Make predictions on new data  
df_test = df_test.drop(columns=['ID'])
dt_pred_new = dt.predict(df_test)  
rf_pred_new = rf.predict(df_test)  
gb_pred_new = gb.predict(df_test)

In [14]:
# Combine the predictions of the base models into a single feature matrix  
X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new))  
  
# Make a prediction using the meta-model  
y_new_pred = meta_model.predict(X_new_meta)    

In [15]:
df_test_result = df_test.copy()
df_test_result['coppaRisk_pred'] = y_new_pred.astype(bool)
#df_test_result['coppaRisk_proba'] = y_new_proba

In [16]:
df_test_result.to_csv('submission.csv', index=False)

### Handling Imbalanced Class dengan Bagging dan Undersampling w/o SMOTE

hasilnya all 0s

In [6]:
from sklearn.model_selection import train_test_split

X = df_train.drop(columns=['coppaRisk'])
y = df_train['coppaRisk']

#split dengan stratified
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [8]:
## 1. Data Preparation with Undersampling
# First apply undersampling to balance the classes
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)  # 2:1 ratio
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

In [31]:
## 2. Base Models with Balanced Training
# Train base models on the resampled data
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

In [18]:
# Wrap base models in BalancedBagging for additional imbalance handling
dt_bag = BalancedBaggingClassifier(
    estimator=dt,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

rf_bag = BalancedBaggingClassifier(
    estimator=rf,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

gb_bag = BalancedBaggingClassifier(
    estimator=gb,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

# Fit the bagged models
dt_bag.fit(X_resampled, y_resampled)
rf_bag.fit(X_resampled, y_resampled)
gb_bag.fit(X_resampled, y_resampled)

BalancedBaggingClassifier(estimator=GradientBoostingClassifier(random_state=42),
                          n_jobs=-1, random_state=42)

In [19]:
## 3. Meta-Feature Generation
# Make predictions on the validation set
#dt_pred = dt_bag.predict(X_val)
#rf_pred = rf_bag.predict(X_val)
#gb_pred = gb_bag.predict(X_val)
dt_pred = dt_bag.predict_proba(X_val)[:, 1]  # Get probabilities for class 1
rf_pred = rf_bag.predict_proba(X_val)[:, 1]
gb_pred = gb_bag.predict_proba(X_val)[:, 1]

# Combine predictions into meta-features
X_val_meta = np.column_stack((dt_pred, rf_pred, gb_pred))

In [20]:
## 4. Meta-Model Training
# Train meta-model on the combined predictions
meta_model = LogisticRegression(random_state = 42)
meta_model.fit(X_val_meta, y_val)

LogisticRegression()

In [21]:
## 5. Prediction Pipeline
# Make predictions on new data
df_test = df_test.drop(columns=['ID'])
#dt_pred_new = dt_bag.predict(df_test)
#rf_pred_new = rf_bag.predict(df_test)
#gb_pred_new = gb_bag.predict(df_test)
dt_pred_new = dt_bag.predict_proba(df_test)[:, 1]
rf_pred_new = rf_bag.predict_proba(df_test)[:, 1]
gb_pred_new = gb_bag.predict_proba(df_test)[:, 1]

In [22]:
# Combine the predictions
X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new))

In [23]:
# Final prediction
#y_new_pred = meta_model.predict(X_new_meta)
y_new_pred = meta_model.predict_proba(X_new_meta)[:, 1]  # Probability of class 1

In [24]:
# Prepare output
df_test_result = df_test.copy()
df_test_result['coppaRisk_pred'] = (y_new_pred > 0.5).astype(bool)  # Convert to binary

In [25]:
df_test_result.to_csv('submission.csv', index=False)

### Handling Imbalanced Class dengan Bagging dan Undersampling + SMOTE

akurasi = 65.436% --> better than smote only

In [15]:
## 2. Base Models with Balanced Training
# Train base models on the resampled data
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

In [16]:
# Wrap base models in BalancedBagging for additional imbalance handling
dt_bag = BalancedBaggingClassifier(
    estimator=dt,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

rf_bag = BalancedBaggingClassifier(
    estimator=rf,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

gb_bag = BalancedBaggingClassifier(
    estimator=gb,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

# Fit the bagged models
dt_bag.fit(oversampled_X, oversampled_Y)
rf_bag.fit(oversampled_X, oversampled_Y)
gb_bag.fit(oversampled_X, oversampled_Y)

BalancedBaggingClassifier(estimator=GradientBoostingClassifier(random_state=42),
                          n_jobs=-1, random_state=42)

In [17]:
## 3. Meta-Feature Generation
# Make predictions on the validation set
#dt_pred = dt_bag.predict(X_val)
#rf_pred = rf_bag.predict(X_val)
#gb_pred = gb_bag.predict(X_val)
dt_pred = dt_bag.predict_proba(X_val)[:, 1]  # Get probabilities for class 1
rf_pred = rf_bag.predict_proba(X_val)[:, 1]
gb_pred = gb_bag.predict_proba(X_val)[:, 1]

# Combine predictions into meta-features
X_val_meta = np.column_stack((dt_pred, rf_pred, gb_pred))

In [18]:
## 4. Meta-Model Training
# Train meta-model on the combined predictions
meta_model = LogisticRegression(random_state = 42)
meta_model.fit(X_val_meta, y_val)

LogisticRegression(random_state=42)

In [21]:
## 5. Prediction Pipeline
# Make predictions on new data
#df_test = df_test.drop(columns=['ID'])
#dt_pred_new = dt_bag.predict(df_test)
#rf_pred_new = rf_bag.predict(df_test)
#gb_pred_new = gb_bag.predict(df_test)
dt_pred_new = dt_bag.predict_proba(df_test)[:, 1]
rf_pred_new = rf_bag.predict_proba(df_test)[:, 1]
gb_pred_new = gb_bag.predict_proba(df_test)[:, 1]

In [22]:
# Combine the predictions
X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new))

In [23]:
# Final prediction
#y_new_pred = meta_model.predict(X_new_meta)
y_new_pred = meta_model.predict_proba(X_new_meta)[:, 1]  # Probability of class 1

In [24]:
# Prepare output
df_test_result = df_test.copy()
df_test_result['coppaRisk_pred'] = (y_new_pred > 0.5).astype(bool)  # Convert to binary

In [40]:
df_test_result.to_csv('submission.csv', index=False)

In [25]:
from sklearn.metrics import roc_auc_score
print("ROC AUC Score:", roc_auc_score(y_val, meta_model.predict_proba(X_val_meta)[:, 1]))

ROC AUC Score: 0.9819018186984939


### Handling Imbalanced Class dengan Bagging dan Undersampling + SMOTE, Membagi Data dengna Stratified K-Fold Cross-Validation

skor = 65.995%

In [15]:
## 2. Base Models with Balanced Training
# Train base models on the resampled data
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

In [16]:
# Wrap base models in BalancedBagging for additional imbalance handling
dt_bag = BalancedBaggingClassifier(
    estimator=dt,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

rf_bag = BalancedBaggingClassifier(
    estimator=rf,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

gb_bag = BalancedBaggingClassifier(
    estimator=gb,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

In [17]:
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
#from sklearn.metrics import roc_auc

skf = StratifiedKFold(n_splits = 10, #5
                      random_state = 42, 
                      shuffle=True)

# Lists to store predictions and true labels across all folds
all_meta_features = []
all_y_true = []

#rocauc_scores = []

for train_index, val_index in skf.split(X,y):
    X_train = X.iloc[train_index, :] 
    X_val = X.iloc[val_index, :] 
    y_train = y[train_index]
    y_val = y[val_index] 

    sm = SMOTE(sampling_strategy='minority', random_state=42)
    
    # Fit SMOTE model to generate the data.
    oversampled_X, oversampled_Y = sm.fit_resample(X_train, y_train)

    # Fit the bagged model
    dt_bag.fit(oversampled_X, oversampled_Y)
    rf_bag.fit(oversampled_X, oversampled_Y)
    gb_bag.fit(oversampled_X, oversampled_Y)

    #Make predictions on the validation set
    dt_pred = dt_bag.predict_proba(X_val)[:, 1]  # Get probabilities for class 1
    rf_pred = rf_bag.predict_proba(X_val)[:, 1]
    gb_pred = gb_bag.predict_proba(X_val)[:, 1]

    # Combine predictions into meta-features
    X_val_meta = np.column_stack((dt_pred, rf_pred, gb_pred))
    all_meta_features.append(X_val_meta)
    all_y_true.append(y_val)

    #Evaluate the model
    #rocauc = rocauc_scores(y_test, y_pred)
    #rocauc_scores.append(accuracy)
    #print(f'Fold ROC AUC: {rocauc}')

# Print average accuracy accross all folds
#print(f'\nAverage Accuracy: {sum(rocauc_scores) / len(rocauc_scores)}')
# Combine predictions and labels from all folds
X_meta = np.vstack(all_meta_features)  # Shape: (n_samples, n_base_models)
y_meta = np.concatenate(all_y_true)    # Shape: (n_samples,)

# Train meta-model on the full set of meta-features
meta_model = LogisticRegression(random_state=42)
meta_model.fit(X_meta, y_meta)

print("Meta-model trained successfully!")


C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File

Meta-model trained successfully!


In [18]:
# Generate base model predictions
dt_pred_new = dt_bag.predict_proba(df_test)[:, 1]
rf_pred_new = rf_bag.predict_proba(df_test)[:, 1]
gb_pred_new = gb_bag.predict_proba(df_test)[:, 1]

In [19]:
# Combine into meta-features
X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new))

In [20]:
# Final prediction (probability of class 1)
y_new_pred = meta_model.predict_proba(X_new_meta)[:, 1]

In [21]:
# --- 5. Prepare Output ---
df_test_result = df_test.copy()
df_test_result['coppaRisk_pred'] = (y_new_pred > 0.5).astype(bool)  # Binary output
df_test_result.to_csv('submission.csv', index=False)

In [22]:
from sklearn.metrics import roc_auc_score
print("ROC AUC Score:", roc_auc_score(y_val, meta_model.predict_proba(X_val_meta)[:, 1]))

ROC AUC Score: 0.8607685009487667


## Trial 3 (Model Stacking dengan base model KNNClassifier, RFClassifier, GaussianNB dan meta model LogisticRegression)

skor = 53.2%

In [9]:
import numpy as np
import warnings
from mlxtend.classifier import StackingClassifier
from sklearn import datasets
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.neighbors import KNeighborsClassifier

In [12]:
## 2. Base Models with Balanced Training
# Train base models on the resampled data
knn = KNeighborsClassifier(n_neighbors=1)
rf = RandomForestClassifier(random_state=42)
gn = GaussianNB()

In [15]:
# Wrap base models in BalancedBagging for additional imbalance handling
knn_bag = BalancedBaggingClassifier(
    estimator=knn,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

rf_bag = BalancedBaggingClassifier(
    estimator=rf,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

gn_bag = BalancedBaggingClassifier(
    estimator=gn,
    sampling_strategy='auto',
    replacement=False,
    random_state=42,
    n_jobs=-1
)

# Fit the bagged models
knn_bag.fit(oversampled_X, oversampled_Y)
rf_bag.fit(oversampled_X, oversampled_Y)
gn_bag.fit(oversampled_X, oversampled_Y)

BalancedBaggingClassifier(estimator=GaussianNB(), n_jobs=-1, random_state=42)

In [16]:
## 3. Meta-Feature Generation
# Make predictions on the validation set
#dt_pred = dt_bag.predict(X_val)
#rf_pred = rf_bag.predict(X_val)
#gb_pred = gb_bag.predict(X_val)
knn_pred = knn_bag.predict_proba(X_val)[:, 1]  # Get probabilities for class 1
rf_pred = rf_bag.predict_proba(X_val)[:, 1]
gn_pred = gn_bag.predict_proba(X_val)[:, 1]

# Combine predictions into meta-features
X_val_meta = np.column_stack((knn_pred, rf_pred, gn_pred))

In [17]:
## 4. Meta-Model Training
# Train meta-model on the combined predictions
meta_model = LogisticRegression(random_state = 42)
meta_model.fit(X_val_meta, y_val)

LogisticRegression(random_state=42)

In [18]:
## 5. Prediction Pipeline
# Make predictions on new data
df_test = df_test.drop(columns=['ID'])
#dt_pred_new = dt_bag.predict(df_test)
#rf_pred_new = rf_bag.predict(df_test)
#gb_pred_new = gb_bag.predict(df_test)
knn_pred_new = knn_bag.predict_proba(df_test)[:, 1]
rf_pred_new = rf_bag.predict_proba(df_test)[:, 1]
gn_pred_new = gn_bag.predict_proba(df_test)[:, 1]

In [19]:
# Combine the predictions
X_new_meta = np.column_stack((knn_pred_new, rf_pred_new, gn_pred_new))

In [20]:
# Final prediction
#y_new_pred = meta_model.predict(X_new_meta)
y_new_pred = meta_model.predict_proba(X_new_meta)[:, 1]  # Probability of class 1

In [21]:
# Prepare output
df_test_result = df_test.copy()
df_test_result['coppaRisk_pred'] = (y_new_pred > 0.5).astype(bool)  # Convert to binary

df_test_result.to_csv('submission.csv', index=False)

## Trial 4 (Model Stacking dengan SVM dan XGB)

In [27]:
from sklearn.ensemble import VotingClassifier

ensemble = VotingClassifier(
    estimators=[
        ('svm', SVC(class_weight='balanced', probability=True)),
        ('xgb', XGBClassifier(scale_pos_weight=6304/696))
    ],
    voting='soft',  # Gabungkan probabilitas
    n_jobs=-1
)

## Trial 5 (Model Stacking dengan Neural Network sebagai Base Model dan SVM/logistic regression/xgboost sebagai Meta-Model)

In [51]:
from keras.models import Sequential, load_model
from keras.layers import Dense

In [45]:
# --- 1. Neural Network Base Models ---
def create_nn_model(input_dim, n_classes):
    model = Sequential()
    model.add(Dense(25, input_dim=input_dim, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [46]:
import os

# create directory for models
os.makedirs('models', exist_ok=True)

In [52]:
# Train and save 5 neural networks
n_members = 5
for i in range(n_members):
    model = create_nn_model(input_dim=X_train.shape[1], n_classes=len(np.unique(y_train)))
    model.fit(X_train, to_categorical(y_train), epochs=50, verbose=0)
    model.save(f'models/model_{i+1}.h5')

C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in th

In [ ]:
# define stacked model from multiple member input models
def define_stacked_model(members):
	# update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
	# define multi-headed input
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	hidden = Dense(10, activation='relu')(merge)
	output = Dense(3, activation='softmax')(hidden)
	model = Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='model_graph.png')
	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# encode output data
	inputy_enc = to_categorical(inputy)
	# fit model
	model.fit(X, inputy_enc, epochs=300, verbose=0)
 
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)

In [ ]:
#Implementation

# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
# define ensemble model
stacked_model = define_stacked_model(members)
# fit stacked model on test dataset
fit_stacked_model(stacked_model, testX, testy)
# make predictions and evaluate
yhat = predict_stacked_model(stacked_model, testX)
yhat = argmax(yhat, axis=1)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

### Logistic Regression

In [8]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from keras.models import load_model  # This was missing
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [8]:
!pip uninstall tensorflow


In [ ]:
pip install tensorflow==2.19.0

In [23]:
!pip install --upgrade adapt

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.8 MB 5.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/12.8 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.8 MB 3.6 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.8 MB 3.1 MB/s eta 0:00:04
   ----------- ---------------------------- 3.7/12.8 MB 3.6 MB/s eta 0:00:03
   -------------- ------------------------- 4.7/12.8 MB 3.7 MB/s eta 0:00:03
   ----------------- ---------------------- 5.5/12.8 MB 3.7 MB/s eta 0:00:02
   ------------------ --------------------- 6.0/12.8 MB 3.7 MB/s eta 0:00:02
   ------------------- -------------------- 6.3/12.8 MB 3.6 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/12.8 MB 3.2 MB/s eta 0:0

In [26]:
!pip install scikeras

In [10]:
import scikeras

ModuleNotFoundError: No module named 'scikeras'

In [9]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.base import clone

# --- 1. Create sklearn-compatible Keras models ---
def create_nn_sklearn(input_dim, n_classes):
    def create_model():
        model = Sequential()
        model.add(Dense(25, input_dim=input_dim, activation='relu'))
        model.add(Dense(n_classes, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    return KerasClassifier(build_fn=create_model, epochs=50, verbose=0)

# --- 2. Cross-Validation Setup ---
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
all_meta_features = []
all_y_true = []
n_members = 5

for train_index, val_index in skf.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Apply SMOTE
    sm = SMOTE(sampling_strategy='minority', random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train)
    
    # Initialize BalancedBagging with NN base estimator
    base_model = create_nn_sklearn(input_dim=X.shape[1], n_classes=len(np.unique(y)))
    bagging = BalancedBaggingClassifier(
        estimator=base_model,
        n_estimators=n_members,
        sampling_strategy='auto',
        replacement=False,
        random_state=42
    )
    
    # Fit on resampled data
    bagging.fit(X_res, y_res)
    
    # Collect predictions from all base estimators
    base_preds = []
    for estimator in bagging.estimators_:
        pred = estimator.predict_proba(X_val)[:, 1]  # Probability of class 1
        base_preds.append(pred)
    
    # Stack predictions
    X_val_meta = np.column_stack(base_preds)
    all_meta_features.append(X_val_meta)
    all_y_true.append(y_val)

# --- 3. Meta-Model Training ---
X_meta = np.vstack(all_meta_features)
y_meta = np.concatenate(all_y_true)

meta_model = LogisticRegression(random_state=42)
meta_model.fit(X_meta, y_meta)

# --- 4. Prediction Function ---
def predict_new_data(bagging_model, meta_model, X_new):
    base_preds = []
    for estimator in bagging_model.estimators_:
        pred = estimator.predict_proba(X_new)[:, 1]
        base_preds.append(pred)
    
    X_new_meta = np.column_stack(base_preds)
    return meta_model.predict_proba(X_new_meta)[:, 1]

ModuleNotFoundError: No module named 'scikeras'

In [ ]:
# --- 5. Prepare Output ---
df_test_result = df_test.copy()
df_test_result['coppaRisk'] = (y_new_pred > 0.5).astype(bool)  # Binary output
df_test_result.to_csv('submission.csv', index=False)

In [ ]:
class NNWrapper:
    def __init__(self, model):
        self.model = model
    
    def predict_proba(self, X):
        return self.model.predict(X)  # Assumes your NN outputs probabilities

# Create BalancedBagging ensembles for each NN
bagged_models = [
    BalancedBaggingClassifier(
        estimator=NNWrapper(model),
        n_estimators=10,
        random_state=42,
        sampling_strategy='auto',
        replacement=False
    ) for model in base_models
]

In [ ]:
# --- 3. Cross-Validation + Meta-Feature Generation ---
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
all_meta_features = []
all_y_true = []

for train_index, val_index in skf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Apply SMOTE to training data
    sm = SMOTE(sampling_strategy='minority', random_state=42)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    
    # Train each bagged model
    for model in bagged_models:
        model.fit(X_train_res, y_train_res)
    
    # Get probabilities from each model
    preds = [model.predict_proba(X_val)[:, 1] for model in bagged_models]
    X_val_meta = np.column_stack(preds)
    
    all_meta_features.append(X_val_meta)
    all_y_true.append(y_val)

In [ ]:
# Combine meta-features
X_meta = np.vstack(all_meta_features)
y_meta = np.concatenate(all_y_true)

# --- 4. Train Meta-Model ---
meta_model = LogisticRegression(random_state=42)
meta_model.fit(X_meta, y_meta)

# --- 5. Prediction Pipeline ---
# Get base model predictions on test data
test_preds = [model.predict_proba(df_test)[:, 1] for model in bagged_models]
X_test_meta = np.column_stack(test_preds)

# Final predictions
y_test_pred = meta_model.predict_proba(X_test_meta)[:, 1]
binary_preds = (y_test_pred > 0.5).astype(int)

# ANN

## Tanpa BalancedBagging

In [2]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

In [3]:
kf = StratifiedKFold(n_splits = 10, #5
                      random_state = 42, 
                      shuffle=True)

In [4]:
def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(X.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

NameError: name 'METRICS' is not defined

In [33]:
# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(X,y)):
    print(f'Fold {fold + 1}')
    
    # Split the data into training and validation sets for this fold
    X_train = X.iloc[train_index, :] 
    X_val = X.iloc[val_index, :] 
    y_train = y[train_index]
    y_val = y[val_index] 
    #X_train_fold, X_val_fold = X_train[train_index], X_train[val_index] //bawaan dari GeekstoGeeks
    #y_train_fold, y_val_fold = y_train[train_index], y_train[val_index] //bawaan dari GeekstoGeeks

    # Fit SMOTE model to generate the data.
    sm = SMOTE(sampling_strategy='minority', random_state=42)
    oversampled_X, oversampled_Y = sm.fit_resample(X_train, y_train)
    
    # Create a new instance of the model for each fold
    model = make_model()
    
    # Train the model on the training fold
    model.fit(oversampled_X, oversampled_Y, epochs=100, batch_size=32, verbose=0)

    # Evaluate
    val_probs = model.predict(X_val)
    val_predictions = np.argmax(val_probs, axis=1)
    
    # ROC-AUC Score (for binary classification)
    if len(np.unique(y_val)) == 2:  # Binary case
        roc_auc = roc_auc_score(y_val, val_probs)
        print(f"ROC AUC Score: {roc_auc:.4f}")

Fold 1


C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ROC AUC Score: 0.8179
Fold 2


C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
ROC AUC Score: 0.8160
Fold 3


...

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ROC AUC Score: 0.8195
Fold 9


C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
ROC AUC Score: 0.8111
Fold 10


C:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
ROC AUC Score: 0.7800


In [34]:
# --- Prepare Output ---
# Assuming you have a test set df_test
test_probs = model.predict(df_test)
y_new_pred = test_probs[:, 1] if test_probs.shape[1] > 1 else test_probs

df_test_result = df_test.copy()
df_test_result['coppaRisk_pred'] = (y_new_pred > 0.5).astype(bool)  # Binary output
df_test_result.to_csv('submission.csv', index=False)
print("\nPredictions saved to submission.csv")

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Predictions saved to submission.csv


### Sama tapi fungsi SKF nya dilengkapi dengan scaling, frequency encoding, imputasi 

In [5]:
df_train = pd.read_csv("train_cleaned_beforesplit (1).csv")
df_test = pd.read_csv("test_cleaned_beforesplit (1).csv")

In [6]:
#split data dengan perbandingan 80:20
from sklearn.model_selection import train_test_split

X = df_train.drop(columns=['coppaRisk'])
y = df_train['coppaRisk']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

In [7]:
skf = StratifiedKFold(n_splits = 10, #5
                      random_state = 42, 
                      shuffle=True)

In [8]:
METRICS = [
      keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
      keras.metrics.MeanSquaredError(name='Brier score'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [9]:
def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(X.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [10]:
cols_to_freqencode = ['developerCountry', 'countryCode', 'primaryGenreName']
cols_to_impute = ['minDownloads', 'maxDownloads']
cols_to_impute_test = ['minDownloads', 'maxDownloads', 'appAge']
cols_to_transform = ['minDownloads', 'maxDownloads']
cols_to_scale = [
    'userRatingCount', 'averageUserRating', 
    'minDownloads', 'maxDownloads',
    'isCorporateEmailScore', 'isCorporateEmail_missing', 
    'appAge'
]

In [11]:
# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(skf.split(X,y)):
    print(f'Fold {fold + 1}')
    
    # Split the data into training and validation sets for this fold
    X_train = X.iloc[train_index, :] 
    X_val = X.iloc[val_index, :] 
    y_train = y[train_index]
    y_val = y[val_index] 
    #X_train_fold, X_val_fold = X_train[train_index], X_train[val_index] //bawaan dari GeekstoGeeks
    #y_train_fold, y_val_fold = y_train[train_index], y_train[val_index] //bawaan dari GeekstoGeeks
    
    #impute missing value
    # Hitung median HANYA dari X_train
    medians = X_train[cols_to_impute].median().to_dict()
    # Imputasi di X_train dan X_val
    for col in cols_to_impute:
        X_train[col].fillna(medians[col], inplace=True)
        X_val[col].fillna(medians[col], inplace=True)

    #Handling outlier
    for col in cols_to_transform:
        # 1. Log transform (hindari log(0) dengan +1)
        X_train[col] = np.log1p(X_train[col])
        X_val[col] = np.log1p(X_val[col])
            
        # 2. Winsorizing (batasi outlier atas & bawah)
        # Hitung limit winsorizing HANYA dari X_train
        l, u = np.percentile(X_train[col], [1, 99])  # Atur persentil sesuai kebutuhan
        X_train[col] = mstats.winsorize(X_train[col], limits=(0.01, 0.01))  # 5% di setiap sisi
        X_val[col] = np.clip(X_val[col], l, u)  # Terapkan limit yang sama ke X_val
            
        # 3. Clip nilai minimum (misal: min=0)
        min_val = X_train[col].min()  # Hitung dari X_train
        X_train[col] = np.clip(X_train[col], min_val, None)
        X_val[col] = np.clip(X_val[col], min_val, None)
            
        # 4. Winsorizing kedua (opsional)
        X_train[col] = mstats.winsorize(X_train[col], limits=(0.01, 0.01))
        X_val[col] = np.clip(X_val[col], *np.percentile(X_train[col], [1, 99]))
    
    # Frequency encoding (hitung frekuensi HANYA dari X_train)
    for col in cols_to_freqencode:
        freq_map = X_train[col].value_counts(normalize=True).to_dict()
        # Apply ke X_train dan X_val
        X_train[f'{col}_freq'] = X_train[col].map(freq_map)
        X_val[f'{col}_freq'] = X_val[col].map(freq_map)
        #Drop kolom original
        X_train.drop(col, axis=1, inplace=True)
        X_val.drop(col, axis=1, inplace=True)
    
    #Scaling
    scaler = StandardScaler()
    for col in cols_to_scale:
        # fit hanya pada X_train
        scaler.fit(X_train[cols_to_scale])
        #transform pada X_train dan X_val
        X_train[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
        X_val[cols_to_scale] = scaler.transform(X_val[cols_to_scale])

    # Fit SMOTE model to generate the data.
    sm = SMOTE(sampling_strategy='minority', random_state=42)
    oversampled_X, oversampled_Y = sm.fit_resample(X_train, y_train)
    
    # Create a new instance of the model for each fold
    model = make_model()
    
    # Train the model on the training fold
    model.fit(oversampled_X, oversampled_Y, epochs=100, batch_size=32, verbose=0)

    # Evaluate
    val_probs = model.predict(X_val)
    val_predictions = np.argmax(val_probs, axis=1)
    
    # ROC-AUC Score (for binary classification)
    if len(np.unique(y_val)) == 2:  # Binary case
        roc_auc = roc_auc_score(y_val, val_probs)
        print(f"ROC AUC Score: {roc_auc:.4f}")

Fold 1


C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\L

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
ROC AUC Score: 0.8343
Fold 2


C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\L

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
ROC AUC Score: 0.8469
Fold 3


...

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
ROC AUC Score: 0.8033
Fold 9


C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\L

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
ROC AUC Score: 0.8594
Fold 10


C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_22736\2202017040.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col].fillna(medians[col], inplace=True)
C:\Users\USER\AppData\L

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
ROC AUC Score: 0.8113


In [12]:
global_median = X['appAge'].median()  # Hitung dari semua data training

In [13]:
global_median

52.9

In [14]:
medians

{'minDownloads': 500.0, 'maxDownloads': 1000.0}

In [15]:
df_test = df_test.drop(columns=['ID'])

In [16]:
#impute missing value pada countryCode dengan GLOBAL
df_test['countryCode'] = df_test['countryCode'].fillna('GLOBAL')

#impute missing value pada cols_to_impute_test = ['minDownloads', 'maxDownloads', 'appAge']
## minDownloads dan maxDownloads
df_test[cols_to_impute] = df_test[cols_to_impute].fillna(medians)
## appAge
df_test['appAge'] = df_test['appAge'].fillna(global_median)

In [17]:
#handling outlier pada kolom minDownloads dan maxDownloads
# Simpan parameter di luar K-Fold
transform_params = {
    'log': True,  # Asumsi pakai log1p
    'winsorize_limits': (0.01, 0.01),  # Sama seperti di K-Fold
    'percentiles': {'minDownloads': None, 'maxDownloads': None},  # Akan diisi
    'min_values': {'minDownloads': None, 'maxDownloads': None}    # Akan diisi
}

# Hitung parameter dari SELURUH data training (X)
for col in ['minDownloads', 'maxDownloads']:
    # a. Hitung percentiles untuk winsorizing (1% dan 99%)
    transform_params['percentiles'][col] = np.percentile(X[col], [1, 99])
    
    # b. Hitung nilai minimum untuk clipping
    transform_params['min_values'][col] = X[col].min()

for col in cols_to_transform:
    # 1. Log transform
    if transform_params['log']:
        df_test[col] = np.log1p(df_test[col])
    
    # 2. Winsorizing (gunakan percentiles dari X)
    l, u = transform_params['percentiles'][col]
    df_test[col] = np.clip(df_test[col], l, u)
    
    # 3. Clip minimum (gunakan nilai dari X)
    min_val = transform_params['min_values'][col]
    df_test[col] = np.clip(df_test[col], min_val, None)
    
    # 4. Winsorizing kedua (opsional)
    df_test[col] = np.clip(df_test[col], *transform_params['percentiles'][col])

In [18]:
#frequency encoding
for col in cols_to_freqencode:
    df_test[f'{col}_freq'] = df_test[col].map(freq_map).fillna(0)

In [19]:
# standardscaler
df_test[cols_to_scale] = scaler.transform(df_test[cols_to_scale])

In [23]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   userRatingCount                   3000 non-null   float64
 1   isCorporateEmailScore             3000 non-null   float64
 2   appAge                            3000 non-null   float64
 3   averageUserRating                 3000 non-null   float64
 4   appDescriptionBrandSafetyRating   3000 non-null   int64  
 5   minDownloads                      3000 non-null   float64
 6   maxDownloads                      3000 non-null   float64
 7   deviceType_GLOBAL                 3000 non-null   bool   
 8   deviceType_connected-tv/ott       3000 non-null   bool   
 9   deviceType_smartphone             3000 non-null   bool   
 10  deviceType_tablet                 3000 non-null   bool   
 11  deviceType_undetermined           3000 non-null   bool   
 12  terms_

In [20]:
df_test.drop(['developerCountry', 'countryCode', 'primaryGenreName'], axis=1, inplace=True)

In [21]:
# --- Prepare Output ---
# Assuming you have a test set df_test
test_probs = model.predict(df_test)
y_new_pred = test_probs[:, 1] if test_probs.shape[1] > 1 else test_probs

df_test_result = df_test.copy()
df_test_result['coppaRisk'] = (y_new_pred > 0.5).astype(bool)  # Binary output
df_test_result.to_csv('submission.csv', index=False)
print("\nPredictions saved to submission.csv")

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

Predictions saved to submission.csv


## Dengan BalancedBagging

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

In [ ]:
kf = StratifiedKFold(n_splits = 10, #5
                      random_state = 42, 
                      shuffle=True)

In [ ]:
def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(X.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [ ]:
n_estimators = 10
models = []
bagging_scores = []

In [ ]:
# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(X,y)):
    print(f'Fold {fold + 1}')
    
    # Split the data into training and validation sets for this fold
    X_train = X.iloc[train_index, :] 
    X_val = X.iloc[val_index, :] 
    y_train = y[train_index]
    y_val = y[val_index] 
    #X_train_fold, X_val_fold = X_train[train_index], X_train[val_index] //bawaan dari GeekstoGeeks
    #y_train_fold, y_val_fold = y_train[train_index], y_train[val_index] //bawaan dari GeekstoGeeks

    # Fit SMOTE model to generate the data.
    sm = SMOTE(sampling_strategy='minority', random_state=42)
    oversampled_X, oversampled_Y = sm.fit_resample(X_train, y_train)
    
    # Create a new instance of the model for each fold
    model = make_model()
    
    # Train the model on the training fold
    model.fit(oversampled_X, oversampled_Y, epochs=100, batch_size=32, verbose=0)

    # Evaluate
    val_probs = model.predict(X_val)
    val_predictions = np.argmax(val_probs, axis=1)
    
    # ROC-AUC Score (for binary classification)
    if len(np.unique(y_val)) == 2:  # Binary case
        roc_auc = roc_auc_score(y_val, val_probs)
        print(f"ROC AUC Score: {roc_auc:.4f}")

In [12]:
#define the model and metrics
METRICS = [
      keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
      keras.metrics.MeanSquaredError(name='Brier score'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [13]:
EPOCHS = 100
BATCH_SIZE = 2048

def early_stopping():
 return tf.keras.callbacks.EarlyStopping(
    monitor='val_prc',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [14]:
model = make_model()
model.summary()

NameError: name 'train_features' is not defined